<a href="https://colab.research.google.com/github/dolu143/Radha/blob/main/ORB_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from smartapi import SmartConnect
import credentials
import functions
import time
import datetime
import talib as ta
import pandas as pd
import numpy as np
import json

In [ ]:
exch_seg = 'NFO'
timeframe = 'FIFTEEN_MINUTE'
fromdate = '2023-06-01 09:15'
todate = '2023-06-28 15:30'
script_profit = {}
quantity = 100
initial_value = 20000

obj= functions.login()
functions.intializeSymbolTokenMap()
tokenInfo = functions.getTokenInfo('NFO','OPTIDX','NIFTY',18700,'CE').iloc[0]
print(tokenInfo)
#symbol  = tokenInfo['symbol']
script  = tokenInfo['symbol']
token = tokenInfo['token']
lot = int(tokenInfo['lotsize'])
print(script, token, lot)

In [ ]:
try:
  hist_data = functions.histcandle(exch_seg,token,timeframe,fromdate,todate)
  if hist_data != None:
    columns = ["date", "open", "high", "low", "close", "volume"]
    df = pd.DataFrame(hist_data["data"], columns = columns)
    print (df)
    if not df.empty:
      order = ""
      totalPnl = 0
      pnl = 0
      exit = 0
      params={}
      result=''
      outtime = ''
      intime = ''
      df1 = pd.DataFrame(columns = ['date', 'intime', 'entry', 'result','outtime','exit', 'PnL', 'total PnL' ])

      for i in df.index:
        ltp=df['close'][i]
        time= df["date"][i][11:16]
        date = df['date'][i][0:10]
        if (time == "09:30") and (order == "buy" or order == ""):
          order = "sell"
          intime = time
          entry = max(df["high"].loc[i-15:i])
          target = round(entry*0.7, 1)
          stoploss = round(entry*1.1, 1)
          params[script] = {'date':date,'intime':intime,"entry":entry,"result":result,'outtime':outtime,'exit':exit,'PnL':pnl,'total PnL':totalPnl}
          #print(f"\n{script} Sold at Rs. {entry} time: {df['date'][i]} ")
          #print (params[script])


        if (time > "09:30") and (order == "sell"):
          #print(df["low"][i])
          if len(params) != 0:

            if (target > int(df["low"][i])) and (time < "13:30"):
              order = "buy"
              outtime = time
              exit = df["low"][i]
              pnl = ((entry - exit)* quantity)
              totalPnl = int(totalPnl + pnl)
              params[script] = {'date':date,'intime':intime,"entry":entry,"result":result,'outtime':outtime,'exit':exit,'PnL':pnl,'total PnL':totalPnl}
              params[script]["result"] = 'Target'
              df1 = df1.append(params[script], ignore_index = True)
              #print(df1)
              #print(f'Target on {df["date"][i][11:-9]} for {script} at Rs.{df["low"][i]}, Profit: {round(pnl,2)}')

            if (stoploss < int(df["high"][i])) and (time < "13:30"):
              order = "buy"
              outtime = time
              exit = df["high"][i]
              pnl = ((entry - exit)* quantity)
              totalPnl = int(totalPnl + pnl)
              params[script] = {'date':date,'intime':intime,"entry":entry,"result":result,'outtime':outtime,'exit':exit,'PnL':pnl,'total PnL':totalPnl}
              params[script]["result"] = 'SL'
              df1 = df1.append(params[script], ignore_index = True)
              #print(df1)
              #print(f'Stoploss on {df["date"][i][11:-9]} for {script} at {df["high"][i]}, Loss: {round(pnl,2)}')

            if (df["date"][i][11:-9] == "13:30"):
              order = "buy"
              outtime = time
              exit = df["low"][i]
              pnl = ((entry - exit)* quantity)
              totalPnl = int(totalPnl + pnl)
              params[script] = {'date':date,'intime':intime,"entry":entry,"result":result,'outtime':outtime,'exit':exit,'PnL':pnl,'total PnL':totalPnl}
              params[script]["result"] = 'Time'
              df1 = df1.append(params[script], ignore_index = True)
              #print(df1)
              #print(f'Exit on {df["date"][i][11:-9]} for {script} at Rs.{df["low"][i]}, PnL: {round(pnl,2)}')

        if len(df) == i+1:
          initial_value = initial_value + totalPnl
          script_profit[script] = totalPnl
          #print(f"\n {script_profit}")
  else:
    print("No Historic Data")

  print(f"\n{script_profit}, Final Value : {initial_value}")
  print(df1)

except Exception as e:
  print("Historic Api failed: {}".format(e.message))


In [ ]:
functions.logout()